In [3]:
import math
import string
import numpy as np
import queue as q

# Define special board cells 
Activate = "A"
Illegal = ""

class Board:
  def __init__(self, arr, activate, illegal):
    self.mBoard = np.array(arr)
    self.CurrentPos = self.find(activate)
    self.IllegalPos = self.find(illegal)
    self.buttonPos = self.getButtonsMap()    # map of button positions, for example {"7" : [0, 0], "8" : [0,1] ... }    
    
  def move(self, stepVert, stepHorz):
    arrow = ""
    if stepVert < 0:
        arrow = "^"
        self.CurrentPos = [self.CurrentPos[0] - 1, self.CurrentPos[1]]
    elif stepVert > 0:
        arrow = "v"
        self.CurrentPos = [self.CurrentPos[0] + 1, self.CurrentPos[1]]
    elif stepHorz < 0:
        arrow = "<"
        self.CurrentPos = [self.CurrentPos[0], self.CurrentPos[1] - 1]
    elif stepHorz > 0:
        arrow = ">"
        self.CurrentPos = [self.CurrentPos[0], self.CurrentPos[1] + 1]
    return arrow
                    
    
  def press(self, toPos):
    illegalAlongVertMove = [toPos[0], self.CurrentPos[1]]
    illegalAlongHorzMove = [self.CurrentPos[0], toPos[1]]
    moveHorzFirst = (illegalAlongVertMove == self.IllegalPos)
    moveVertFirst = (illegalAlongHorzMove == self.IllegalPos)    

    directions = []
    moveVert = toPos[0] - self.CurrentPos[0]
    moveHorz = toPos[1] - self.CurrentPos[1]
    #optimize not up! - left and right first
    while moveVert != 0 or moveHorz !=0:
        if (not moveVertFirst) and moveHorz < 0: #move left
            moveHorz += 1
            directions += [self.move(0, -1)]

        elif (not moveHorzFirst) and moveVert > 0: #movedown
            moveVert -= 1
            directions += [self.move(1, 0)]
            
        elif (not moveHorzFirst) and moveVert < 0: #moveup
            moveVert += 1
            directions += [self.move(-1, 0)]
            
        elif (not moveVertFirst) and moveHorz > 0: #move right
            moveHorz -= 1
            directions += [self.move(0, 1)]
                
        if moveVert == 0:
            moveVertFirst = False
        if moveHorz == 0:
            moveHorzFirst = False
       
    directions += ["A"]
    return directions


  def getButtonsMap(self):
    buttonsMap = {}
    for vert in range(self.mBoard.shape[0]):
        for horz in range(self.mBoard.shape[1]):
            buttonsMap[str(self.mBoard[vert, horz])] = [vert, horz]
    return buttonsMap

        
  def find(self, elem):
    for vert in range(self.mBoard.shape[0]):
        for horz in range(self.mBoard.shape[1]):
            if self.mBoard[vert,horz] == elem: return [vert, horz]
    return [-1, -1]


  def print(self):
    for vert in range(self.mBoard.shape[0]):
        rowStr = ""
        for horz in range(self.mBoard.shape[1]):
            if (self.CurrentPos == np.array([vert, horz])).all(): 
                rowStr = rowStr + str(self.mBoard[vert, horz]) + "*" + "\t" 
            else:
                rowStr = rowStr + str(self.mBoard[vert, horz]) + "\t" 
        print(rowStr)    

In [4]:
#######################################################################
# Very Efficient memory wise and Performant
# Latest depth first with step count at level and next step caches
#######################################################################
def calculateCostFromLength(code, length):
    num = int(code[0 : len(code)-1])
    return num * length


def addToCache(key, value, cache):
    if key not in cache:
        cache[key] = value


def countDepthFirst(code, keypads, currentlyAt, cacheNextStep, cacheStepCountAtLevel, predecessorTuples):
    keypad = keypads[currentlyAt]
    levelCount = 0

    for x in range(len(code)):
        #get from/to tuple for the next steps cache
        currButton = str(keypad.mBoard[keypad.CurrentPos[0], keypad.CurrentPos[1]])
        targetButton = code[x]
        fromToTuple = (currButton, targetButton)

        #if in cacheNextStep --> get the cached path for next level
        if fromToTuple in cacheNextStep:
            path = cacheNextStep[fromToTuple]
            currButton = targetButton
            keypad.CurrentPos = keypad.buttonPos[targetButton]
        #if not in cacheNextStep --> get path for next level and cache it
        else:
            path = keypad.press(keypad.buttonPos[targetButton])
            cacheNextStep[fromToTuple] = path
            # this is a leaf node so add its count for lvl 0 so we don't have to recalculate it 
            fromToAtLeafNodeTutuple = (fromToTuple, 0)
            addToCache(fromToAtLeafNodeTutuple, len(path), cacheStepCountAtLevel)

        #print ("Current level: " + str(currentlyAt) + ", tuple: " + str(fromToTuple) + ", path: " + str(path) + ", predecessorTuples: " + str(predecessorTuples))
        # we're currently levelsFromMax away from leaf nodes. Create a Tutuple key to map count: (('fromKey', 'toKey'), lvlsFromMax) for the cacheStepCountAtLevel cache
        levelsFromMax = len(keypads) - 1 - (currentlyAt)
        fromToAtCurrentLevelTutuple = (fromToTuple, levelsFromMax)
        
        #if we already calculated this count before, use it and skip desceding deeper
        if fromToAtCurrentLevelTutuple in cacheStepCountAtLevel:
            levelCount += cacheStepCountAtLevel[fromToAtCurrentLevelTutuple]
            #print (" -->Found in cache, Current level: " + str(currentlyAt) + ", tuple: " + str(fromToAtCurrentLevelTutuple))
            continue
        
        nextLevelCount = 0
        if currentlyAt == len(keypads) - 1:
            nextLevelCount = len(path)
        else:
            # call next level recursivly after appending latest predecessor tuple
            nextLevelPredecessorTuples = predecessorTuples.copy()
            nextLevelPredecessorTuples.append(fromToTuple)
            nextLevelCount = countDepthFirst(path, keypads, currentlyAt+1, cacheNextStep, cacheStepCountAtLevel, nextLevelPredecessorTuples)
        
        levelCount += nextLevelCount
        
        #cache this result --> the cache entry is for (max - current) level deep
        addToCache(fromToAtCurrentLevelTutuple, nextLevelCount, cacheStepCountAtLevel)

    return levelCount

def countForCode(code, numDirectional, cacheNextStep, cacheStepCountAtLevel):
    keypads = []
    predecessorTuples = []

    # append the numeric keypad and all directional keypads
    keypads.append(Board([["7", "8", "9"], ["4", "5", "6"], ["1", "2", "3"], [Illegal, "0", Activate]], Activate, Illegal))
    for x in range(numDirectional):
        keypads.append(Board([[Illegal, "^", Activate], ["<", "v", ">"]], Activate, Illegal))

    count = countDepthFirst(code, keypads, 0, cacheNextStep, cacheStepCountAtLevel, predecessorTuples)

    return count

def processMultipleCodes(codes, numDirectional):
    totalCost = 0
    cacheNextStep = {}
    cacheStepCountAtLevel = {}
    count = 0
    for code in codes:
        count = countForCode(code, numDirectional, cacheNextStep, cacheStepCountAtLevel)
        codeCost = calculateCostFromLength(code, count)
        totalCost += codeCost
        #print("Processed " + code + ", this code cost= " + str(codeCost))
    #print("processMultipleCodes total cost: " + str(totalCost))
    return totalCost

In [ ]:
codes = []
with open('input/Day21.txt', 'r') as file:
    for line in file:
        codes.append(line.replace('\n', ''))
print(codes)

#%load_ext memory_profiler
#%timeit cost = processMultipleCodes(codes,2960) 
#%timeit cost = processMultipleCodes(codes, 25)
 
cost = processMultipleCodes(codes, 25) 
print(cost)

['780A', '539A', '341A', '189A', '682A']
216668579770346


%timeit cost = processMultipleCodes(codes, 25) 
1.98 ms ± 5.11 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

%memit cost = processMultipleCodes(codes, 25)
peak memory: 95.24 MiB, increment: 0.86 MiB
2.06 ms ± 18.8 μs per run over 100 runs



%timeit cost = processMultipleCodes(codes,2960)
637 ms ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

%memit cost = processMultipleCodes(codes, 2967)
peak memory: 113.34 MiB, increment: 19.03 MiB
330 ms ± 10.9 ms per run over 100 runs